In [133]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
from Data_Generation import generate_data
import numpy as np

In [134]:
'''
STEP 0: utils 
'''
class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, list_IDs, labels):
        'Initialization'
        self.labels = labels
        self.list_IDs = list_IDs

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label
        X = torch.as_tensor(ID)
        y = self.labels[index]

        return X, y

def create_data(data_type, data_out):
    # Generating synthetic data
    x_train, y_train, _ = generate_data(n = 10000, data_type = data_type, seed = 0, out = data_out)  
    x_test,  y_test,  _  = generate_data(n = 10000,  data_type = data_type, seed = 1,  out = data_out)
    return torch.Tensor(x_train), torch.Tensor(y_train), torch.Tensor(x_test), torch.Tensor(y_test)

def bernoulli_sampling (prob):
    samples = torch.distributions.binomial.Binomial(1, prob).sample()
    return samples

def my_loss(sel_prob,dis_prob,val_prob, y_pred,y_true):
    m = nn.Softmax(dim=1)
    dis_prob=m(dis_prob)
    val_prob=m(val_prob)
    y_true=y_true.type(torch.FloatTensor)
    Reward1=torch.sum(y_true*torch.log(dis_prob+1e-8),dim=1)
    Reward2=torch.sum(y_true*torch.log(val_prob+1e-8),dim=1)
    Reward=Reward1-Reward2
    loss1 = Reward*torch.sum(sel_prob*torch.log(y_pred+1e-8)+(1-sel_prob)*torch.log(1-y_pred+1e-8),dim=1)-0.1*y_pred.mean(dim=1)
    loss = torch.mean(-loss1)
    return loss

In [135]:
'''
STEP 1: LOADING DATASET
'''
x_train, y_train, x_test, y_test= create_data('Syn3', 'Y')
training_set = Dataset(x_train, y_train)
testing_set=Dataset(x_test, y_test)

In [136]:
'''
STEP 2: MAKING DATASET ITERABLE
'''
batch_size = 10
n_iters = 10000
num_epochs = n_iters / (len(x_train) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=training_set,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=testing_set, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [137]:
'''
STEP 3: CREATE MODEL CLASS
'''
# SAME AS LINEAR REGRESSION!!
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim,output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 200)
        self.selu1=nn.SELU()
        self.bn1 = nn.BatchNorm1d(200)
        self.fc2=nn.Linear(200,200)
        self.selu2=nn.SELU()
        self.bn2 = nn.BatchNorm1d(200)
        self.fc3=nn.Linear(200,output_dim)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.selu1(out)
        out = self.bn1(out)
        out = self.fc2(out)
        out = self.selu2(out)
        out = self.bn2(out)
        out = self.fc3(out)
        return out

class Actor(nn.Module):
    def __init__(self, input_dim):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(input_dim, 100)
        self.selu1=nn.SELU()
        self.fc2=nn.Linear(100,100)
        self.selu2=nn.SELU()
        self.fc3=nn.Linear(100,input_dim)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.selu1(out)
        out = self.fc2(out)
        out = self.selu2(out)
        out = self.fc3(out)
        out = self.sigmoid(out)
        return out

In [138]:
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 11
output_dim = 2

base = FeedforwardNeuralNetModel(input_dim, output_dim)
discriminator = FeedforwardNeuralNetModel(input_dim, output_dim)
actor = Actor(input_dim)


#######################
#  USE GPU FOR MODEL  #
#######################

if torch.cuda.is_available():
    base.cuda()
    discriminator.cuda()
    actor.cuda()

In [139]:
'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()



In [140]:
'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.001
optimizer = torch.optim.Adam([
    {'params': base.parameters()},{'params': discriminator.parameters()},{'params': actor.parameters()}], lr=learning_rate)

In [141]:
'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
for epoch in range(num_epochs):
    for i, (inp, labels) in enumerate(train_loader):
        
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        if torch.cuda.is_available():
            inp = Variable(inp.view(-1, 11).cuda())
            labels = Variable(labels.cuda().type(torch.LongTensor))
        else:
            inp = Variable(inp.view(-1, 11))
            labels = Variable(labels.type(torch.LongTensor))
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
                
        # outputs
        gen_prob = actor(inp)
        sel_prob = bernoulli_sampling(gen_prob)
        dis_prob = discriminator(inp*sel_prob)
        val_prob = base(inp)
        
        # losses
        d_loss=criterion(dis_prob,labels[:,1])
        v_loss=criterion(val_prob,labels[:,1])
        s_loss=my_loss(sel_prob,dis_prob,val_prob, gen_prob,labels)
        
        # backwards
        d_loss.backward(retain_graph=True)
        v_loss.backward(retain_graph=True)
        s_loss.backward()
        
        optimizer.step()            
        

        iter += 1
        
        if iter % 1000 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            correct2 = 0
            # Iterate through test dataset
            for inp, labels in test_loader:
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                if torch.cuda.is_available():
                    inp = Variable(inp.view(-1, 11).cuda())
                    labels = Variable(labels.cuda().type(torch.LongTensor))
                else:
                    inp = Variable(inp.view(-1, 11))
                    labels = Variable(labels.type(torch.LongTensor))    
                
                # Forward pass only to get logits/output
                val_prob = base(inp)
                y_pred = actor(inp)
                
                sel_prob = bernoulli_sampling(gen_prob)
                
                dis_prob = discriminator(inp*sel_prob)                
                
                # Get predictions from the maximum value
                _, predicted = torch.max(val_prob.data, 1)
                _, predicted2 = torch.max(dis_prob.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                # Total correct predictions
                correct += (predicted.cpu() == labels[:,1].cpu()).sum()
                correct2 += (predicted2.cpu() == labels[:,1].cpu()).sum()
            
            accuracy = 100 * correct / total
            accuracy2 = 100 * correct2 / total
            
            # Print Loss
            print('Iteration: {}. Base-Loss: {}. Accuracy: {}'.format(iter, v_loss, accuracy))
            print('Iteration: {}. Critic-Loss: {}. Accuracy: {}'.format(iter, d_loss, accuracy2))
            #print(y_pred)

Iteration: 1000. Base-Loss: 0.2701040208339691. Accuracy: 77
Iteration: 1000. Critic-Loss: 12.637089729309082. Accuracy: 47
Iteration: 2000. Base-Loss: 0.6399099230766296. Accuracy: 77
Iteration: 2000. Critic-Loss: 2.061149835586548. Accuracy: 46
Iteration: 3000. Base-Loss: 0.6496340036392212. Accuracy: 78
Iteration: 3000. Critic-Loss: 6.120383262634277. Accuracy: 48
Iteration: 4000. Base-Loss: 0.49853724241256714. Accuracy: 78
Iteration: 4000. Critic-Loss: 11.7402982711792. Accuracy: 47
Iteration: 5000. Base-Loss: 0.2888660430908203. Accuracy: 78
Iteration: 5000. Critic-Loss: 9.53486156463623. Accuracy: 47
Iteration: 6000. Base-Loss: 0.24689121544361115. Accuracy: 78
Iteration: 6000. Critic-Loss: 7.790999412536621. Accuracy: 48
Iteration: 7000. Base-Loss: 0.3755154609680176. Accuracy: 78
Iteration: 7000. Critic-Loss: 6.976513862609863. Accuracy: 45
Iteration: 8000. Base-Loss: 0.49812135100364685. Accuracy: 78
Iteration: 8000. Critic-Loss: 8.766035079956055. Accuracy: 44
Iteration: 900

In [29]:
'''
Model parameters
'''
print(model.parameters)
print(len(list(model.parameters())))

# Hidden Layer Parameters
print(list(model.parameters())[0].size())
# FC 1 Bias Parameters
print(list(model.parameters())[1].size())
# FC 2 Parameters
print(list(model.parameters())[2].size())
# FC 2 Bias Parameters
print(list(model.parameters())[3].size())


<bound method Module.parameters of FeedforwardNeuralNetModel(
  (fc1): Linear(in_features=11, out_features=500, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=500, out_features=500, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=500, out_features=2, bias=True)
)>
6
torch.Size([500, 11])
torch.Size([500])
torch.Size([500, 500])
torch.Size([500])


In [30]:
save_model = False
if save_model is True:
    torch.save(model.estate_dict(),'base_model.pkl')

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 11 and 2 in dimension 2 at c:\programdata\miniconda3\conda-bld\pytorch_1532505617613\work\aten\src\th\generic/THTensorMath.cpp:3616